In [1]:
from tools import stream
import requests
import pandas as pd

In [2]:
def get_book(instrument, tmp_typ, bucketNumber):
    if tmp_typ == 'positions':
        typ = 'positionBook'
    elif tmp_typ == 'orders':
        typ = 'orderBook'
    else:
        return 'Wrong type'
    
    response = requests.get(
        url = 'https://api-fxpractice.oanda.com/v3/instruments/{}/{}'.format(instrument, typ),
        headers = {
            'Authorization' : 'Bearer ' + open('key', 'r').read(),
        }
    ).json()

    if 'errorMessage' in response.keys():
        return response['errorMessage']
    else:
        bucketPrices = []
        bucketDict = {}
        for bucket in response[typ]['buckets']:
            bucketPrices.append(float(bucket['price']))
            bucketDict[float(bucket['price'])] = {
                "long": bucket['longCountPercent'],
                "short": bucket['shortCountPercent'],
            }
        
        returnBuckets = {}
        closestPrice = min(bucketPrices, key=lambda x:abs(x-float(response[typ]['price'])))
        for i in range(bucketNumber+1, 1, -1):
            priceToLookUp = round(closestPrice + (float(response[typ]['bucketWidth']) * i),4)
            returnBuckets[priceToLookUp] = {
                'longs': bucketDict[priceToLookUp]['long'],
                'shorts': bucketDict[priceToLookUp]['short'],
            }
            
        returnBuckets[closestPrice] = {
            'longs': bucketDict[closestPrice]['long'],
            'shorts': bucketDict[closestPrice]['short'],
        }
            
        for i in range(1, bucketNumber+1):
            priceToLookUp = round(closestPrice - (float(response[typ]['bucketWidth']) * i),4)
            returnBuckets[priceToLookUp] = {
                'longs': bucketDict[priceToLookUp]['long'],
                'shorts': bucketDict[priceToLookUp]['short'],
            }
        
        return response

In [3]:
get_book('EUR_USD', 'orders', 3)

{'orderBook': {'instrument': 'EUR_USD',
  'time': '2019-04-16T18:20:00Z',
  'unixTime': '1555438800',
  'price': '1.12863',
  'bucketWidth': '0.00050',
  'buckets': [{'price': '0.00000',
    'longCountPercent': '0.0273',
    'shortCountPercent': '0.0638'},
   {'price': '0.00050',
    'longCountPercent': '0.0000',
    'shortCountPercent': '0.0030'},
   {'price': '0.00100',
    'longCountPercent': '0.0091',
    'shortCountPercent': '0.0395'},
   {'price': '0.00500',
    'longCountPercent': '0.0030',
    'shortCountPercent': '0.0061'},
   {'price': '0.00550',
    'longCountPercent': '0.0030',
    'shortCountPercent': '0.0000'},
   {'price': '0.00800',
    'longCountPercent': '0.0030',
    'shortCountPercent': '0.0000'},
   {'price': '0.01000',
    'longCountPercent': '0.0121',
    'shortCountPercent': '0.0091'},
   {'price': '0.01200',
    'longCountPercent': '0.1276',
    'shortCountPercent': '0.0000'},
   {'price': '0.01750',
    'longCountPercent': '0.0000',
    'shortCountPercent': '0

In [1]:
from tools import stream, get_book

In [2]:
get_book('EUR_USD', 'orders', 3)

{1.1305: {'longs': '0.0907', 'shorts': '0.1626'},
 1.13: {'longs': '0.0594', 'shorts': '0.0782'},
 1.1295: {'longs': '0.0563', 'shorts': '0.1220'},
 1.1285: {'longs': '0.0876', 'shorts': '0.1595'},
 1.128: {'longs': '0.1877', 'shorts': '0.0844'},
 1.1275: {'longs': '0.4254', 'shorts': '0.4066'},
 1.127: {'longs': '0.4285', 'shorts': '0.2877'}}